Document를 보고 질문에 대한 정답 document 인덱스를 만들어야 함.

In [10]:
import os
import pandas as pd
from tqdm import tqdm

import openai
from langchain_community.vectorstores import Milvus

from langchain_openai import OpenAIEmbeddings, ChatOpenAI




In [11]:
# 임베딩 모델이름
company = 'OPENAI'
embedding_models = 'text-embedding-3-large'
header_include = True
table_include = True
split_yn = False
documnet_name = "2024 IONIQ5"

In [12]:
exp_name = f"{company}_{embedding_models}_h{header_include}_t{table_include}_s{split_yn}"
exp_dir = '../experiment/result/' + exp_name

embedding_result_path = exp_dir + '/embeddings_pc_chunking.parquet'
df = pd.read_parquet(embedding_result_path)

In [13]:
df = df.drop_duplicates('doc_id')[['doc_id', 'h1', 'h2', 'h3', 'doc_contents', 'table_contents']]

In [14]:
df

,doc_id,h1,h2,h3,doc_contents,table_contents
0,0,1. 안내 및 차량 정보,구동 모터 번호,,\n구동 모터 번호 \n2WD / 4WD\n4WD\n구동 모터 번호는 차량 밑에서 ...,[]
1,1,1. 안내 및 차량 정보,모터룸,,\n모터룸\n이 그림은 실제 차량과 다를 수 있습니다.,[]
2,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...
3,3,1. 안내 및 차량 정보,자기 인증 라벨,,\n자기 인증 라벨\n자기 인증 라벨은 차량이 대한민국 자동차 관리법령에 맞게 제작...,[]
4,4,1. 안내 및 차량 정보,전구의 용량,,\n전구의 용량\nLED(Light-Emitting Diode): 발광소자,[| | 전구의 용량 | 전구의 용량 ...
...,...,...,...,...,...,...
1400,469,전기차 시작하기,전기 자동차 충전 종류,충전에 대한 정보,충전에 대한 정보\n충전에 대한 정보\n충전의 종류에는 공공 충전소의 전용 충전기를...,[]
1403,470,전기차 시작하기,전기 자동차 충전 표시등,,\n전기 자동차 충전 표시등\n구동용(고전압) 배터리 충전 시 차량 외부에서충전량을...,[]
1404,471,전기차 시작하기,전기 자동차 충전 표시등,충전 상태 표시등,충전 상태 표시등\n충전 도어\n\n전면 범퍼 중앙,[| | 충전 표시등 동작 | 충전량[%] |\n|---:|:-----...
1406,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...


# LLM을 활용한 Evaluation Test Set 생성

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI


## GPT 3.5 XML Context(QA creation at the same time)

In [ ]:
llm = ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'], temperature=0, model="gpt-3.5-turbo")

In [ ]:
def make_cluade_context(row):
    chapter = row['h1']
    majorheading = row['h2']
    minorheading = row['h3']
    context = row['doc_contents']
    tables = '\n'.join(list(map(lambda x: f"<table {x[0]}>\n" + x[1] + f"\n</table {x[0]}>\n", enumerate(row['table_contents']))))
    context_string = f"""
    <document>
        <chapter>
            {chapter}
        </chapter>
        <majorheading>
            {majorheading}
        </majorheading> 
        <minorheading>
            {minorheading}
        </minorheading> 
        <context>
            {context}
        </context> 
        <tables>
            {tables}
        </tables> 
    </document>
    """
    
    return context_string

In [ ]:
claude_df = df.reset_index(drop=True)
claude_df['claude_context'] = claude_df.apply(lambda x: make_cluade_context(x), axis=1).values

In [456]:
retriever_prompt_template = """
Given the following document chunk in XML format, 
generate a diverse set of {num_questions_per_chunk} questions and answer that a general user might ask based on the provided context. 
Consider the general user as someone who:
  - Is a first-time vehicle user.
  - Has limited knowledge about vehicles and may not understand technical terms mentioned in the manual. 
  
<document>
    <chapter>
        {chapter}
    </chapter> 
    <majorheading>
        {majorheading}
    </majorheading> 
    <minorheading>
        {minorheading}
    </minorheading> 
    <context>
        {context}
    </context> 
    <tables> 
        {tables} 
    </tables> 
</document> 

Please adhere to the following guidelines when generating questions:
    1. Questions should be based on the content within the <context> tags and should not include prior knowledge. 
    2. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate questions. 
    3. Generate a diverse set of {num_questions_per_chunk} questions that cover the entire document. 
    4. Each question should not start with "-" and should not include multiple-choice options. 
    5. Please number each question in the generated set of questions. 
    6. Questions should be phrased in a way that is easily understandable for a general user who is a first-time vehicle user and may not be familiar with technical terms. 
    7. Don't say any other sentences other than the questions you've created. The generated questions will be used to evaluate the search accuracy performance of a vector database. Aim to create questions that effectively test the system's ability to retrieve the original document chunk based on user queries. Write in Korean.
    
Please adhere to the following guidelines when generating answer:
    1. Answers to questions should be provided with actual content between the <document> tags provided when creating the question. Please only refer to {chapter}, {majorheading}, {minorheading}, {context}, {tables} in the XML format provided. You should never use prior knowledge.
    2. Please only summarize the answers to the core except for the incidental answers and write them in three sentences.
    
Please adhere to the following guidelines when generating question-answer pair:
    1. Please create the format for generating questions and answers in the format below.
        - {question number}
            - {question}
            - {answer}
    - Example
        '''
        - 19
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        - 20
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        '''

\n\nAssistant:"""


PROMPT_RETRIEVER = PromptTemplate(
    template=retriever_prompt_template,
    input_variables=["chapter", "majorheading", "minorheading", "context", "num_questions_per_chunk"]
)

In [ ]:
gpt_df = claude_df.copy()

In [ ]:
for i, row in tqdm(gpt_df.iterrows()):
    xml_context = row['claude_context']
    
    messages = [
        SystemMessage(content=retriever_prompt_template),
        HumanMessage(content=xml_context)
    ]

    aiMessage=llm.invoke(messages)
    gpt_df.loc[i, 'question'] = aiMessage.content

332it [25:22,  4.59s/it]


In [482]:
gpt_df.to_csv('../evaluation_dataset/qaset_gpt35_xmlcontext_240429.csv', index=False)
gpt_df.to_parquet('../evaluation_dataset/qaset_gpt35_xmlcontext_240429.parquet', index=False)

In [483]:


def process_document(document):
    # 문서를 개행 문자로 분리하여 리스트로 변환
    try:
        lines = document.split('\n')
        
        # 빈 문자열 제거
        lines = [line for line in lines if line.strip()]
        lines = [line.lstrip()[1:] for line in lines]
        lines = [line.lstrip() for line in lines]
        data = []
        for i in list(range(0, len(lines),3)):
            question_num = int(lines[i])
            question = lines[i+1]
            answer = lines[i+2]
            data.append([question_num, question, answer])
        df = pd.DataFrame(data, columns=['질문번호', '질문', '대답'])
    except:
        df = pd.DataFrame([[None,None,None]], columns=['질문번호', '질문', '대답'])
        
    return df

In [484]:
gpt35qaset = pd.DataFrame()
for i,row in gpt_df.iterrows():
    doc_id = row['doc_id']
    cur_qa = process_document(row['question'])
    cur_qa['doc_id'] = doc_id
    gpt35qaset = pd.concat([gpt35qaset,cur_qa], ignore_index=True)

In [ ]:
for doc_id in tqdm(gpt_df.loc[gpt35qaset[gpt35qaset['질문번호'].isna()]['doc_id']]['doc_id']):
    
    xml_context = gpt_df.loc[doc_id, 'claude_context']
    
    messages = [
        SystemMessage(content=retriever_prompt_template),
        HumanMessage(content=xml_context)
    ]

    aiMessage=llm.invoke(messages)
    gpt_df.loc[doc_id, 'question'] = aiMessage.content

100%|██████████| 7/7 [00:28<00:00,  4.09s/it]


In [486]:
gpt35qaset

,질문번호,질문,대답,doc_id
0,1,구동 모터 번호는 어디에서 확인할 수 있나요?,구동 모터 번호는 차량 밑에서 확인할 수 있습니다.,0
1,1,모터룸에 대해 어떤 내용이 포함되어 있나요?,모터룸에 대한 그림은 실제 차량과 다를 수 있다고 언급되어 있습니다.,1
2,1,에어컨 시스템 작업 시 지정된 냉매를 사용해야 하는 이유는 무엇인가요?,지정되지 않은 냉매를 사용하면 냉방 성능이 저하되고 심각한 부품 손상이 발생할 수 ...,2
3,2,에어컨 시스템에서 사용되는 에어컨 냉매의 용량은 얼마인가요?,에어컨 냉매의 용량은 900g±25입니다.,2
4,3,에어컨 시스템에 사용되는 냉동유의 용량은 얼마인가요?,냉동유의 용량은 180±10입니다.,2
...,...,...,...,...
1319,1,완속 충전 시 충전 커넥터 잠금 모드는 어떤 기능을 하는 건가요?,완속 충전 시 충전 커넥터 잠금 모드는 외부인이 충전 커넥터를 충전 인렛으로부터 분...,472
1320,2,충전 커넥터 잠금을 해제하려면 어떤 작업을 해야 하나요?,충전 커넥터 잠금을 해제하기 위해서는 차량 도어 잠금을 해제해야 합니다.,472
1321,3,충전 중에 도어를 잠그면 어떻게 되나요?,충전 중에 도어를 잠그면 충전 커넥터도 즉시 다시 잠기게 됩니다.,472
1322,1,충전이 시작되지 않은 경우 어떤 확인사항을 해야 하나요?,"충전이 시작되지 않는 경우 예약 충전 설정, 충전기의 작동 상태, 클러스터에 표시된...",473


In [492]:
gpt_df[['doc_id', 'h1', 'h2', 'h3', 'doc_contents', 'table_contents']].merge(gpt35qaset.drop('질문번호', axis=1), on='doc_id', how='left')

,doc_id,h1,h2,h3,doc_contents,table_contents,질문,대답
0,0,1. 안내 및 차량 정보,구동 모터 번호,,\n구동 모터 번호 \n2WD / 4WD\n4WD\n구동 모터 번호는 차량 밑에서 ...,[],구동 모터 번호는 어디에서 확인할 수 있나요?,구동 모터 번호는 차량 밑에서 확인할 수 있습니다.
1,1,1. 안내 및 차량 정보,모터룸,,\n모터룸\n이 그림은 실제 차량과 다를 수 있습니다.,[],모터룸에 대해 어떤 내용이 포함되어 있나요?,모터룸에 대한 그림은 실제 차량과 다를 수 있다고 언급되어 있습니다.
2,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...,에어컨 시스템 작업 시 지정된 냉매를 사용해야 하는 이유는 무엇인가요?,지정되지 않은 냉매를 사용하면 냉방 성능이 저하되고 심각한 부품 손상이 발생할 수 ...
3,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...,에어컨 시스템에서 사용되는 에어컨 냉매의 용량은 얼마인가요?,에어컨 냉매의 용량은 900g±25입니다.
4,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...,에어컨 시스템에 사용되는 냉동유의 용량은 얼마인가요?,냉동유의 용량은 180±10입니다.
...,...,...,...,...,...,...,...,...
1319,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,완속 충전 시 충전 커넥터 잠금 모드는 어떤 기능을 하는 건가요?,완속 충전 시 충전 커넥터 잠금 모드는 외부인이 충전 커넥터를 충전 인렛으로부터 분...
1320,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,충전 커넥터 잠금을 해제하려면 어떤 작업을 해야 하나요?,충전 커넥터 잠금을 해제하기 위해서는 차량 도어 잠금을 해제해야 합니다.
1321,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,충전 중에 도어를 잠그면 어떻게 되나요?,충전 중에 도어를 잠그면 충전 커넥터도 즉시 다시 잠기게 됩니다.
1322,473,전기차 시작하기,충전이 시작되지 않은 경우 확인사항,,\n충전이 시작되지 않은 경우 확인사항\n충전기를 차량에 연결한 후 충전이 시작되지...,[],충전이 시작되지 않은 경우 어떤 확인사항을 해야 하나요?,"충전이 시작되지 않는 경우 예약 충전 설정, 충전기의 작동 상태, 클러스터에 표시된..."


In [493]:
gpt_df[['doc_id', 'h1', 'h2', 'h3', 'doc_contents', 'table_contents']].merge(gpt35qaset.drop('질문번호', axis=1), on='doc_id', how='left').to_csv('../evaluation_dataset/evaluation_qaset_240429_gpt35-turbo.csv', index=False)
gpt_df[['doc_id', 'h1', 'h2', 'h3', 'doc_contents', 'table_contents']].merge(gpt35qaset.drop('질문번호', axis=1), on='doc_id', how='left').to_parquet('../evaluation_dataset/evaluation_qaset_240429_gpt35-turbo.parquet', index=False)

## Cluade 수동

In [463]:
retriever_prompt_template = """
 \n\nGiven the following document chunk in XML format, generate a diverse set of {num_questions_per_chunk} questions that a general user might ask based on the provided context.
Consider the general user as someone who:

 - Is a first-time vehicle user.
 - Has limited knowledge about vehicles and may not understand technical terms mentioned in the manual.

<document>
  <chapter>{chapter}</chapter>
  <majorheading>{majorheading}</majorheading>
  <minorheading>{minorheading}</minorheading>
  <context>{context}</context>
  <tables>
    {tables}
  </tables>
</document>
Please adhere to the following guidelines when generating questions:

1. Questions should be based on the content within the <context> tags and should not include prior knowledge.
2. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate questions.
3. Generate a diverse set of {num_questions_per_chunk} questions that cover the entire document.
4. Each question should not start with "-" and should not include multiple-choice options.
5. Please number each question in the generated set of questions.
6. Questions should be phrased in a way that is easily understandable for a general user who is a first-time vehicle user and may not be familiar with technical terms.
7. Don't say any other sentences other than the questions you've created.

The generated questions will be used to evaluate the search accuracy performance of a vector database. Aim to create questions that effectively test the system's ability to retrieve the original document chunk based on user queries.
Write in Korean. 
\n\nAssistant:"""

PROMPT_RETRIEVER = PromptTemplate(
    template=retriever_prompt_template,
    input_variables=["chapter", "majorheading", "minorheading", "context", "tables", "num_questions_per_chunk"]
)

In [246]:
retriever_prompt_template = """
\n\nHuman: Here is the context information, inside <context></context> XML tags.
Please don't make questions with the contents of the table.

<chapter>{chapter}</chapter>Given the chapter name
<majorheading>{majorheading}</majorheading>Given the major heading of chapter.
<minorheading>{minorheading}</minorheading>Given the minor heading of chapter.
<context>{context}</context>Given the context information and not prior knowledge.
<table>{table}</table> Given the table context information formatted markdown and not prior knowledge
generate only questions based on the below query.
You are a Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \quiz/examination.
The questions should be diverse in nature \across the document.
The questions should not contain options, start with "-"
Restrict the questions to the context information provided.
Write in Korean. 

\n\nAssistant:"""

PROMPT_RETRIEVER = PromptTemplate(
    template=retriever_prompt_template,
    input_variables=["chapter", "majorheading", "minorheading", "context", "num_questions_per_chunk"]
)

In [18]:
generation_prompt_template = """
Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>
Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.
    - Skip the preamble
    - Use three sentences maximum and keep the answer concise.
    - If the answer is not in the context, just say "Could not find answer in given contexts."
    - The each answers should start with "-"
    - Answer in Korean.
Question:
{question}
Answer:"""

PROMPT_GENERATION = PromptTemplate(
    template=generation_prompt_template,
    input_variables=["context", "question"]
)

In [78]:
generation_prompt_template = """
 Given the following question and associated context and table information in XML format, generate a single answer to the question based on the provided context and table(s).
<question>{question}</question>
<context>{context}</context>
<tables>
  {tables}
</tables>
Please adhere to the following guidelines when generating the answer:

1. Be courteous and polite.
2. Only answer the question if you can find the answer in the context with certainty.
3. Skip the preamble.
4. Use three sentences maximum and keep the answer concise.
5. If the answer is not in the context, just say "Could not find answer in given contexts."
6. Note that there may be multiple questions provided. Each question has a number associated with it. Please use the same number when generating the answer for that question, and prefix each answer with the corresponding question number (e.g., "1. ...", "2. ...", ..., "N. ...").
7. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate answer.
8. Answer in Korean.

The generated answer will be used to evaluate the performance of a RAG (Retrieval-Augmented Generation) system in answering questions based on the provided context and table(s).
 """
 
PROMPT_GENERATION = PromptTemplate(
    template=generation_prompt_template,
    input_variables=["context", "tables", "question"]
)

In [79]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


chain1 = LLMChain(llm=llm,prompt=PROMPT_RETRIEVER,output_key="question",verbose=False)
chain2 = LLMChain(llm=llm,prompt=PROMPT_GENERATION,output_key="answer", verbose=False)

chain = SequentialChain(chains=[chain1,chain2],
                        input_variables=["chapter", "majorheading", "minorheading", "context", "tables", "num_questions_per_chunk"],
                        output_variables=['context', 'tables', 'question','answer'],verbose=False)

In [80]:
df['question'] = ''
df['answer'] = ''

In [81]:
def make_qa_from_pandas(pdf):
    qaset = pdf.copy()
    for i, row in tqdm(qaset.iterrows()):
        chapter = row['h1']
        majorheading = row['h2']
        minorheading = row['h3']
        context = row['doc_contents']
        tables = '\n'.join(list(map(lambda x: f"<table {x[0]}>\n" + x[1] + f"\n</table {x[0]}>\n", enumerate(row['table_contents']))))
        if len(context)<512:
            num_questions_per_chunk = 1
        elif (len(context)>=512) and (len(context)<1024):
            num_questions_per_chunk = 2
        else:
            num_questions_per_chunk = 3
        
        result = chain({'chapter': chapter, 'majorheading': majorheading, 'minorheading': minorheading, 'context': context,'tables': tables, 'num_questions_per_chunk':num_questions_per_chunk})
        question = result['question']
        answer = result['answer']
            
        qaset.loc[i, 'question'] = question
        qaset.loc[i, 'answer'] = answer
    return qaset

In [87]:
df.reset_index().loc[4:10]

,index,doc_id,h1,h2,h3,doc_contents,table_contents,question,answer
4,4,4,1. 안내 및 차량 정보,전구의 용량,,\n전구의 용량\nLED(Light-Emitting Diode): 발광소자,[| | 전구의 용량 | 전구의 용량 ...,,
5,9,5,1. 안내 및 차량 정보,차대번호 (VIN),,\n차대번호 (VIN)\n차대번호(VIN: Vehicle Identification...,[],,
6,10,6,1. 안내 및 차량 정보,차량 내부,,\n차량 내부\n차량 내부 I\n이 그림은 적용된 사양에 따라 실제 차량과 다를 수...,[],,
7,11,7,1. 안내 및 차량 정보,차량 외부,,\n차량 외부\n차량 외부 I\n이 그림은 실제 차량과 다를 수 있습니다.\n차량 ...,[],,
8,12,8,1. 안내 및 차량 정보,추천 오일 및 용량,,\n추천 오일 및 용량\n 각 시스템의 원활한 작동과 수명을 위해 품질과 성능이 적...,[| | 종 류 | 종 류 | 종 류 | 용량( ℓ ) ...,,
9,16,9,1. 안내 및 차량 정보,타이어 및 휠,,\n타이어 및 휠\n 각 도로의 규정 속도 이내로 주행하십시오. 해발 고도가 높을...,[| | 구 분 | 형 식 | 휠 | 추천 공기...,,
10,18,10,1. 안내 및 차량 정보,타이어 속도 등급,,\n타이어 속도 등급\n아래 표는 승용차 및 소형 승용차에 사용되는 타이어의 속도 ...,[| | 속도 등급 | 최대 속도 |\n|---:|:----...,,


In [100]:
qaset2 = make_qa_from_pandas(df.reset_index(drop=True).loc[101:])

373it [29:33,  4.75s/it]


In [101]:
qaset2

,doc_id,h1,h2,h3,doc_contents,table_contents,question,answer
101,101,5. 편의 장치,디지털 사이드 미러 (DSM),디지털 사이드 미러 작동,디지털 사이드 미러 작동\nDSM 모니터 켜기도어 잠금을 해제하거나 원격 시동을 걸...,[],1. DSM 모니터가 언제 자동으로 꺼지고 켜지나요?,1. DSM 모니터는 시동을 끄고 도어를 잠그거나 시동을 끈 지 7분이 지나면 자동...
102,102,5. 편의 장치,디지털 사이드 미러 (DSM),후진 시 와이드 뷰 전환,후진 시 와이드 뷰 전환\n후진 시 와이드 뷰 전환\n와이드 뷰/일반 뷰\n‘R’(...,[],1. 디지털 사이드 미러가 후진 시 와이드 뷰로 전환되는 방법은 무엇인가요?\n2....,1. 후진 시 디지털 사이드 미러가 와이드 뷰로 전환되는 방법은 'R'로 변속하고 ...
103,103,5. 편의 장치,디지털 센터 미러 (DCM),,\n디지털 센터 미러 (DCM)\n디지털 센터 미러는 차량 후방의 카메라를 이용하여...,[],1. 디지털 센터 미러의 주요 기능은 무엇인가요?\n2. 디지털 센터 미러를 조작할...,1. 디지털 센터 미러의 주요 기능은 차량 후방의 카메라를 이용하여 후방 시야를 확...
104,104,5. 편의 장치,디지털 센터 미러 (DCM),디스플레이 설정 (디지털 미러 모드),디스플레이 설정 (디지털 미러 모드)\n디스플레이 설정 (디지털 미러 모드)\n디지...,[| | 습니다. | 습니다. |\n...,1. 디지털 센터 미러의 밝기를 조절하는 방법은 무엇인가요?\n2. 디지털 센터 미...,1. 메뉴 버튼을 눌러 디지털 센터 미러의 밝기와 표시화면의 상하 각도를 조절할 수...
105,105,5. 편의 장치,디지털 센터 미러 (DCM),모드 변경 방법,모드 변경 방법 \n모드 변경 방법 \n[A] 디지털 모드[B] 일반 모드시동 ‘O...,[],1. 디지털 센터 미러의 모드를 변경하는 방법은 무엇인가요?,1. 디지털 센터 미러의 모드를 변경하는 방법은 모드 선택 레버를 당기거나 밀어서 ...
...,...,...,...,...,...,...,...,...
469,469,전기차 시작하기,전기 자동차 충전 종류,충전에 대한 정보,충전에 대한 정보\n충전에 대한 정보\n충전의 종류에는 공공 충전소의 전용 충전기를...,[],1. 전기차를 충전할 때 사용할 수 있는 세 가지 충전 방법은 무엇인가요?\n2. ...,"1. 완속 충전, 급속 충전, 휴대용 충전 방법을 사용할 수 있습니다.\n2. 급속..."
470,470,전기차 시작하기,전기 자동차 충전 표시등,,\n전기 자동차 충전 표시등\n구동용(고전압) 배터리 충전 시 차량 외부에서충전량을...,[],1. 전기 자동차 충전 시 차량 외부에서 어떻게 충전량을 확인할 수 있나요?,1. 충전 표시등을 통해 전기 자동차의 외부에서 충전량을 확인할 수 있습니다.
471,471,전기차 시작하기,전기 자동차 충전 표시등,충전 상태 표시등,충전 상태 표시등\n충전 도어\n\n전면 범퍼 중앙,[| | 충전 표시등 동작 | 충전량[%] |\n|---:|:-----...,1. 전기차 충전 상태를 나타내는 표시등은 어디에 위치하고 있나요?,1. 전기차 충전 상태를 나타내는 표시등은 전면 범퍼 중앙에 위치하고 있습니다.
472,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,다음 문서 조각에 대해 사용자가 할 수 있는 다양한 질문 세트를 생성하십시오.\n\...,1. 충전 중에 충전 커넥터를 분리하지 않으면 자동으로 잠금 상태가 되는 시간은 1...


In [102]:
qaset2.to_csv('../evaluation_dataset/200_eval_dataset.csv', index=False)
qaset2.to_parquet('../evaluation_dataset/200_eval_dataset.parquet', index=False)

In [97]:
qaset.to_csv('../evaluation_dataset/100_eval_dataset.csv', index=False)
qaset.to_parquet('../evaluation_dataset/100_eval_dataset.parquet', index=False)

In [104]:
df

,doc_id,h1,h2,h3,doc_contents,table_contents,question,answer
0,0,1. 안내 및 차량 정보,구동 모터 번호,,\n구동 모터 번호 \n2WD / 4WD\n4WD\n구동 모터 번호는 차량 밑에서 ...,[],,
1,1,1. 안내 및 차량 정보,모터룸,,\n모터룸\n이 그림은 실제 차량과 다를 수 있습니다.,[],,
2,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...,,
3,3,1. 안내 및 차량 정보,자기 인증 라벨,,\n자기 인증 라벨\n자기 인증 라벨은 차량이 대한민국 자동차 관리법령에 맞게 제작...,[],,
4,4,1. 안내 및 차량 정보,전구의 용량,,\n전구의 용량\nLED(Light-Emitting Diode): 발광소자,[| | 전구의 용량 | 전구의 용량 ...,,
...,...,...,...,...,...,...,...,...
1400,469,전기차 시작하기,전기 자동차 충전 종류,충전에 대한 정보,충전에 대한 정보\n충전에 대한 정보\n충전의 종류에는 공공 충전소의 전용 충전기를...,[],,
1403,470,전기차 시작하기,전기 자동차 충전 표시등,,\n전기 자동차 충전 표시등\n구동용(고전압) 배터리 충전 시 차량 외부에서충전량을...,[],,
1404,471,전기차 시작하기,전기 자동차 충전 표시등,충전 상태 표시등,충전 상태 표시등\n충전 도어\n\n전면 범퍼 중앙,[| | 충전 표시등 동작 | 충전량[%] |\n|---:|:-----...,,
1406,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,,


## Cluade 수동 작성..

In [464]:


def process_document(document):
    # 문서를 개행 문자로 분리하여 리스트로 변환
    lines = document.split('\n')
    
    # 빈 문자열 제거
    lines = [line for line in lines if line.strip()]
    lines = [line.lstrip()[1:] for line in lines]
    lines = [line.lstrip() for line in lines]
    data = []
    for i in list(range(0, len(lines),3)):
        question_num = int(lines[i])
        question = lines[i+1]
        answer = lines[i+2]
        data.append([question_num, question, answer])
    df = pd.DataFrame(data, columns=['질문번호', '질문', '대답'])
        
    return df

In [201]:
claude_qa = pd.DataFrame()

In [184]:
claude_df = df.reset_index(drop=True)

In [185]:
claude_df['claude_context'] = claude_df.apply(lambda x: make_cluade_context(x), axis=1).values

In [192]:
claude_df

,doc_id,h1,h2,h3,doc_contents,table_contents,question,answer,claude_context
0,0,1. 안내 및 차량 정보,구동 모터 번호,,\n구동 모터 번호 \n2WD / 4WD\n4WD\n구동 모터 번호는 차량 밑에서 ...,[],,,\n <document>\n <chapter>\n ...
1,1,1. 안내 및 차량 정보,모터룸,,\n모터룸\n이 그림은 실제 차량과 다를 수 있습니다.,[],,,\n <document>\n <chapter>\n ...
2,2,1. 안내 및 차량 정보,에어컨 시스템,,"\n에어컨 시스템\n에어컨 관련 작업 시, 반드시 지정된 냉매를 사용하십시오.지정되...",[| | 에어컨 시스템 | 에어컨 시스템 | 에어컨 시스템 |\n|...,,,\n <document>\n <chapter>\n ...
3,3,1. 안내 및 차량 정보,자기 인증 라벨,,\n자기 인증 라벨\n자기 인증 라벨은 차량이 대한민국 자동차 관리법령에 맞게 제작...,[],,,\n <document>\n <chapter>\n ...
4,4,1. 안내 및 차량 정보,전구의 용량,,\n전구의 용량\nLED(Light-Emitting Diode): 발광소자,[| | 전구의 용량 | 전구의 용량 ...,,,\n <document>\n <chapter>\n ...
...,...,...,...,...,...,...,...,...,...
469,469,전기차 시작하기,전기 자동차 충전 종류,충전에 대한 정보,충전에 대한 정보\n충전에 대한 정보\n충전의 종류에는 공공 충전소의 전용 충전기를...,[],,,\n <document>\n <chapter>\n ...
470,470,전기차 시작하기,전기 자동차 충전 표시등,,\n전기 자동차 충전 표시등\n구동용(고전압) 배터리 충전 시 차량 외부에서충전량을...,[],,,\n <document>\n <chapter>\n ...
471,471,전기차 시작하기,전기 자동차 충전 표시등,충전 상태 표시등,충전 상태 표시등\n충전 도어\n\n전면 범퍼 중앙,[| | 충전 표시등 동작 | 충전량[%] |\n|---:|:-----...,,,\n <document>\n <chapter>\n ...
472,472,전기차 시작하기,충전 커넥터 잠금,완속 충전 시 충전 커넥터 잠금 모드,완속 충전 시 충전 커넥터 잠금 모드\n완속 충전 시 충전 커넥터 잠금 모드\n충전...,[| | 커넥터 잠금 | 동작 | 동작 | 동작 |...,,,\n <document>\n <chapter>\n ...


In [329]:
print("""
Given the following document chunk in XML format, 
generate a diverse set of {num_questions_per_chunk} questions that a general user might ask based on the provided context. 
Consider the general user as someone who:
  - Is a first-time vehicle user.
  - Has limited knowledge about vehicles and may not understand technical terms mentioned in the manual. 
  
<document>
    <chapter>
        {chapter}
    </chapter> 
    <majorheading>
        {majorheading}
    </majorheading> 
    <minorheading>
        {minorheading}
    </minorheading> 
    <context>
        {context}
    </context> 
    <tables> 
        {tables} 
    </tables> 
</document> 

Please adhere to the following guidelines when generating questions:
    1. Questions should be based on the content within the <context> tags and should not include prior knowledge. 
    2. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate questions. 
    3. Generate a diverse set of {num_questions_per_chunk} questions that cover the entire document. 
    4. Each question should not start with "-" and should not include multiple-choice options. 
    5. Please number each question in the generated set of questions. 
    6. Questions should be phrased in a way that is easily understandable for a general user who is a first-time vehicle user and may not be familiar with technical terms. 
    7. Don't say any other sentences other than the questions you've created. The generated questions will be used to evaluate the search accuracy performance of a vector database. Aim to create questions that effectively test the system's ability to retrieve the original document chunk based on user queries. Write in Korean.
    
Please adhere to the following guidelines when generating answer:
    1. 질문에 대한 대답은 질문을 만들때 제공한 <document> 태그 사이에 실제 콘텐츠가 제공되어야 합니다. 제공된 XML 형식에는 {chapter}, {majorheading}, {minorheading}, {context}, {tables}에서만 참조해서 답해주세요. 사전지식은 절대 사용하면 안됩니다.
    2. 지금부터 대답은 부수적인 대답은 제외하고 핵심으로만 추려서 세문장 내로 작성해줘.
    
Please adhere to the following guidelines when generating question-answer pair:
    1. 질문과 그에 대한 대답을 다음의 내용을 고려해서 생성한 각 질문에 대한 질문-답 쌍으로 작성해주세요.
    2. 질문과 답을 생성하는 형식은 아래 형식으로 생성해주세요.
        - {question number}
            - {question}
            - {answer}
    - 예시
        '''
        - 19
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        - 20
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        '''
  
\n\nAssistant:

""")


Given the following document chunk in XML format, 
generate a diverse set of {num_questions_per_chunk} questions that a general user might ask based on the provided context. 
Consider the general user as someone who:
  - Is a first-time vehicle user.
  - Has limited knowledge about vehicles and may not understand technical terms mentioned in the manual. 
  
<document>
    <chapter>
        {chapter}
    </chapter> 
    <majorheading>
        {majorheading}
    </majorheading> 
    <minorheading>
        {minorheading}
    </minorheading> 
    <context>
        {context}
    </context> 
    <tables> 
        {tables} 
    </tables> 
</document> 

Please adhere to the following guidelines when generating questions:
    1. Questions should be based on the content within the <context> tags and should not include prior knowledge. 
    2. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate questions. 
    3. Generate a

In [465]:
gpt_df.loc[gpt35qaset[gpt35qaset['질문번호'].isna()]['doc_id']]

,doc_id,h1,h2,h3,doc_contents,table_contents,question,answer,claude_context
81,81,5. 편의 장치,도난 경보 장치,경계 및 경보 상태 해제,경계 및 경보 상태 해제\n경계 및 경보 상태 해제\n스마트 키의 도어 잠금 해제 ...,[],\n- 1\n - 차량의 경계 및 경보 상태를 해제하는 방법은 무엇인가요?\n...,,\n <document>\n <chapter>\n ...
133,133,5. 편의 장치,스티어링 휠 (조향 핸들),스티어링 휠 (조향 핸들 열선),스티어링 휠 (조향 핸들 열선)\n스티어링 휠 (조향 핸들 열선)\n 시동이 걸린 ...,[],\n- 1\n - 스티어링 휠 열선을 켜려면 어떻게 해야 하나요?\n- 2\n...,,\n <document>\n <chapter>\n ...
167,167,5. 편의 장치,유리창 열림/닫힘 장치,원격 유리창 열림/닫힘 기능 (리모트 윈도우 컨트롤),원격 유리창 열림/닫힘 기능 (리모트 윈도우 컨트롤)\n원격 유리창 열림/닫힘 기능...,[],\n- 1\n - 원격 유리창 열림/닫힘 기능을 사용할 때 주의해아 할 점은 ...,,\n <document>\n <chapter>\n ...
217,217,5. 편의 장치,현대 디지털 키,디지털 키 (스마트폰),디지털 키 (스마트폰) \n디지털 키 (스마트폰) \n현대 디지털 키는 내부 사용성...,[],\n- 1\n - 현대 디지털 키를 사용할 수 있는 스마트폰 모델은 어디에서 ...,,\n <document>\n <chapter>\n ...
291,291,7. 운전자 보조,내비게이션 기반 스마트 크루즈 컨트롤 (NSCC),내비게이션 기반 스마트 크루즈 컨트롤 작동,내비게이션 기반 스마트 크루즈 컨트롤 작동\n작동 조건내비게이션 기반 스마트 크루즈...,[],\n- 1\n - 내비게이션 기반 스마트 크루즈 컨트롤이 작동 대기 상태가 되...,,\n <document>\n <chapter>\n ...
296,296,7. 운전자 보조,서라운드 뷰 모니터 (SVM),서라운드 뷰 모니터 작동,서라운드 뷰 모니터 작동 \n서라운드 뷰 모니터 작동 \n주차/뷰 버튼(1)을 누르...,[],\n- 1\n - 서라운드 뷰 모니터를 켜려면 어떤 버튼을 눌러야 하나요?\n...,,\n <document>\n <chapter>\n ...
375,375,8. 비상시 응급 조치,주행 중 차량 고장 시 대처 방법,고장 자동차 정지 표지판 (안전삼각대),고장 자동차 정지 표지판 (안전삼각대)\n사용 방법\n 고장 자동차 정지 표지판(안...,[],\n- 1\n - 고장 자동차 정지 표지판(안전 삼각대)은 어디에 보관해야 하...,,\n <document>\n <chapter>\n ...


In [481]:
gpt_df.loc[375, 'question'] = '''
* 150
   * 고장 자동차 정지 표지판(안전 삼각대)은 언제 사용하나요?
      * 차량 고장 등의 사유로 고속도로 등에서 차량을 운행할 수 없을 때 도로교통법에 따라 고장 자동차 정지 표지판(안전 삼각대)을 설치해야 합니다.

* 151
   * 고장 자동차 정지 표지판(안전 삼각대)은 어디에 보관해야 하나요?
      * 고장 자동차 정지 표지판(안전 삼각대)은 항상 사용하기 쉬운 위치, 예를 들면 화물칸 내부나 고객이 원하는 곳에 보관하고, 그 위치를 확인해 두어야 합니다.

* 152
   * 고장 자동차 정지 표지판(안전 삼각대)은 어떻게 설치하나요?
      * 케이스에서 고장 자동차 정지 표지판(안전 삼각대)을 꺼내 좌우 날개를 올려 삼각형으로 맞춘 후 연결 홈에 고정하고, 아래쪽 네 개의 다리를 펼쳐 차량 뒤에서 접근하는 자동차 운전자가 확인할 수 있는 위치에 넘어지지 않게 세워야 합니다. 설치 시에는 통행 차량에 주의하여 안전하게 해야 합니다.
'''

In [480]:
doc_id = 375
claude_df.loc[doc_id, 'claude_context']


'\n    <document>\n        <chapter>\n            8.  비상시 응급 조치\n        </chapter>\n        <majorheading>\n            주행 중 차량 고장 시 대처 방법\n        </majorheading> \n        <minorheading>\n            고장 자동차 정지 표지판 (안전삼각대)\n        </minorheading> \n        <context>\n            고장 자동차 정지 표지판 (안전삼각대)\n사용 방법\n 고장 자동차 정지 표지판(안전 삼각대)은 차량 출고 시 이 취급설명서와 함께 키트 박스에 넣어져 지급됩니다. 고장 자동차 정지 표지판(안전 삼각대)을 항상 사용하기 쉬운 위치(화물칸내 또는 고객이 원하는 위치)에 보관하십시오. 차량 고장 등의 사유로 고속도로 등에서 차량을 운행할 수 없을 때 고장 자동차 정지 표지판(안전 삼각대)을 설치하도록 도로교통법에서 규정하고 있습니다. 고장 자동차 정지 표지판(안전 삼각대)을 항상 휴대하고 보관 위치를 확인하십시오.\n\n설치 방법\n케이스에서 고장 자동차 정지 표지판(안전 삼각대)을 꺼내 좌우 날개를 올려 삼각형으로 맞춘 후 서로 교차하게 연결 홈에 고정하십시오.고장 자동차 정지 표지판(안전 삼각대) 아래쪽에 있는 네 개의 다리를 좌우로 펼쳐 차량 뒤에서 접근하는 자동차의 운전자가 고장 차량을 확인할 수 있는 위치에 넘어지지 않게 세우십시오.\n고장 자동차 정지 표지판(안전 삼각대)을 설치할 때는 통행 차량에 주의하여 안전하게 설치하십시오.\n        </context> \n        <tables>\n            \n        </tables> \n    </document>\n    '

In [323]:
qa = """
* 118
   * 차에서 내릴 때 어린이와 관련하여 주의해야 할 점은 무엇인가요?
      * 차에서 내릴 때는 어린이를 절대로 차 안에 혼자 남겨두지 마십시오. 어린이가 차내 안전 장비를 잘못 만져 사고가 발생할 수 있고, 차 내부 온도가 너무 높거나 낮아져 위험할 수 있습니다.

* 119
   * 어린이가 차 안에서 안전하게 탑승하려면 어떻게 해야 하나요?
      * 어린이는 보호자와 함께 뒷좌석에 태우고 안전벨트나 적절한 보호장치를 착용시켜야 합니다. 또한 뒷도어의 어린이 보호용 잠금장치를 작동시켜 어린이가 차 안에서 문을 열지 못하도록 해야 합니다. 

* 120
   * 어린이가 적절한 보호장치를 착용하지 않으면 어떤 위험이 있나요?
      * 만약 어린이가 적절한 보호장치를 착용하지 않은 상태에서 급제동이나 충돌 사고가 발생하면 매우 위험합니다. 충격으로 인해 어린이가 차 안에서 튕겨 나가거나 심각한 부상을 입을 수 있습니다.
"""

new_claude_qa = process_document(qa.replace("Copy code*   ", ""))
new_claude_qa['doc_id'] = doc_id
claude_qa = pd.concat([claude_qa,new_claude_qa])

In [324]:
claude_qa.tail(10)

,질문번호,질문,대답,doc_id
0,111,차량 탑승 시 안전벨트 착용이 필요한 이유는 무엇인가요?,안전벨트는 사고 발생 시 탑승자를 좌석에 고정시켜 충격으로 인한 부상이나 사망 위험...,20
1,112,안전벨트를 올바르게 착용하는 방법은 무엇인가요?,안전벨트를 올바르게 착용하려면 먼저 엉덩이를 좌석 가장 안쪽으로 넣고 등을 등받이에...,20
2,113,안전벨트를 착용할 수 없는 어린이의 경우 어떻게 해야 하나요?,3점식 안전벨트를 착용할 수 없는 어린이의 경우에는 뒷좌석에 어린이용 보조 좌석을 ...,20
3,114,임산부나 환자가 안전벨트 착용 시 주의할 점은 무엇인가요?,임산부나 환자의 경우 급정차나 충돌 사고가 발생할 때 안전벨트로 인해 복부 등에 강...,20
0,115,안전벨트 착용 시 주의해야 할 사항은 무엇인가요?,"안전벨트 착용 시 어깨띠를 팔 밑으로 내리거나 등 뒤로 두지 말아야 하며, 목이나 ...",21
1,116,안전벨트 착용을 방해하는 물품에는 어떤 것들이 있나요?,"차단 클립, 안전벨트 스토퍼, 놀이방 매트 등은 안전벨트의 정상적인 착용과 사용을 ...",21
2,117,안전벨트 착용의 중요성과 관련 법규에 대해 설명해 주세요.,안전벨트는 교통사고 발생 시 탑승자의 생명과 신체를 보호하는 데 매우 중요한 역할을...,21
0,118,차에서 내릴 때 어린이와 관련하여 주의해야 할 점은 무엇인가요?,차에서 내릴 때는 어린이를 절대로 차 안에 혼자 남겨두지 마십시오. 어린이가 차내 ...,22
1,119,어린이가 차 안에서 안전하게 탑승하려면 어떻게 해야 하나요?,어린이는 보호자와 함께 뒷좌석에 태우고 안전벨트나 적절한 보호장치를 착용시켜야 합니...,22
2,120,어린이가 적절한 보호장치를 착용하지 않으면 어떤 위험이 있나요?,만약 어린이가 적절한 보호장치를 착용하지 않은 상태에서 급제동이나 충돌 사고가 발생...,22


In [325]:
claude_qa.to_csv('../evaluation_dataset/iomiq5_2024_calude.csv',index=False)
claude_qa.to_parquet('../evaluation_dataset/iomiq5_2024_calude.parquet',index=False)

In [ ]:
retriever_prompt_template = """
Given the following document chunk in XML format, 
generate a diverse set of {num_questions_per_chunk} questions that a general user might ask based on the provided context. 
Consider the general user as someone who:
  - Is a first-time vehicle user.
  - Has limited knowledge about vehicles and may not understand technical terms mentioned in the manual. 
  
<document>
    <chapter>
        {chapter}
    </chapter> 
    <majorheading>
        {majorheading}
    </majorheading> 
    <minorheading>
        {minorheading}
    </minorheading> 
    <context>
        {context}
    </context> 
    <tables> 
        {tables} 
    </tables> 
</document> 

Please adhere to the following guidelines when generating questions:
    1. Questions should be based on the content within the <context> tags and should not include prior knowledge. 
    2. If tables (<table>) are provided within the <tables> tags and their content can be clearly interpreted, use them to generate questions. 
    3. Generate a diverse set of {num_questions_per_chunk} questions that cover the entire document. 
    4. Each question should not start with "-" and should not include multiple-choice options. 
    5. Please number each question in the generated set of questions. 
    6. Questions should be phrased in a way that is easily understandable for a general user who is a first-time vehicle user and may not be familiar with technical terms. 
    7. Don't say any other sentences other than the questions you've created. The generated questions will be used to evaluate the search accuracy performance of a vector database. Aim to create questions that effectively test the system's ability to retrieve the original document chunk based on user queries. Write in Korean.
    
Please adhere to the following guidelines when generating answer:
    1. 질문에 대한 대답은 질문을 만들때 제공한 <document> 태그 사이에 실제 콘텐츠가 제공되어야 합니다. 제공된 XML 형식에는 {chapter}, {majorheading}, {minorheading}, {context}, {tables}에서만 참조해서 답해주세요. 사전지식은 절대 사용하면 안됩니다.
    2. 지금부터 대답은 부수적인 대답은 제외하고 핵심으로만 추려서 세문장 내로 작성해줘.
    
Please adhere to the following guidelines when generating question-answer pair:
    1. 질문과 그에 대한 대답을 다음의 내용을 고려해서 생성한 각 질문에 대한 질문-답 쌍으로 작성해주세요.
    2. 질문과 답을 생성하는 형식은 아래 형식으로 생성해주세요.
        - {question number}
            - {question}
            - {answer}
    - 예시
        '''
        - 19
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        - 20
            - 동승석에 어린이 보조 좌석을 설치하면 어떻게 되나요?
            - 동승석에 어린이 보조 좌석을 설치하면 승객 구분 시스템(OCS)이 이를 감지하고 에어백을 작동하지 않습니다. 하지만 어린이 보조 좌석을 설치한 후 PASSENGER AIR BAG OFF 표시등이 꺼지면 에어백이 작동할 수 있으므로, 즉시 어린이 보조 좌석을 뒷좌석에 설치해야 합니다.
        '''

  
\n\nAssistant:"""


PROMPT_RETRIEVER = PromptTemplate(
    template=retriever_prompt_template,
    input_variables=["chapter", "majorheading", "minorheading", "context", "num_questions_per_chunk"]
)

In [354]:
gpt_df = claude_df.copy()

In [373]:
for i, row in tqdm(gpt_df[84+33+25:].iterrows()):
    xml_context = row['claude_context']
    
    messages = [
        SystemMessage(content=retriever_prompt_template),
        HumanMessage(content=xml_context)
    ]

    aiMessage=llm.invoke(messages)
    gpt_df.loc[i, 'question'] = aiMessage.content

332it [25:22,  4.59s/it]


In [90]:
for i, row in qaset[['doc_contents', 'table_contents', 'question', 'answer']].iterrows():
    print("======DOC CONTENTS=====")
    print(row['doc_contents'])
    for tbl in row['table_contents']:
        print(tbl)
    print("=========QUESTION=========")
    print(row['question'])
    print("==========ANSWER=========")
    print(row['answer'])
    print('=========================================================================================================')

======DOC CONTENTS=====

전구의 용량
LED(Light-Emitting Diode): 발광소자
|    | 전구의 용량   | 전구의 용량                               | 전구의 용량   |
|---:|:--------------|:------------------------------------------|:--------------|
|  0 | 종 류         |                                           |               |
|  1 | 차량 앞       | 전조등                                    | 하향등        |
|  2 |               |                                           | 상향등        |
|  3 |               | 방향 지시등                               |               |
|  4 |               |                                           |               |
|  5 |               | 차폭등                                    |               |
|  6 |               | 주간 주행등                               |               |
|  7 |               | 프론트 가니쉬 램프                        |               |
|  8 |               | 충전 표시등                               |               |
|  9 |               | 프론트 트렁크                             |               |
|

In [250]:
bad_request_index = []
for i, row in tqdm(df.iterrows()):
    chapter = row['h1']
    majorheading = row['h2']
    minorheading = row['h3']
    context = row['doc_contents']
    table_img_urls = row['table_img_urls']
    if not table_img_urls:
        if len(context)<512:
            num_questions_per_chunk = 1
        elif (len(context)>=512) and (len(context)<1024):
            num_questions_per_chunk = 2
        else:
            num_questions_per_chunk = 3
        
        result = chain({'chapter': chapter, 'majorheading': majorheading, 'minorheading': minorheading, 'context': context,'num_questions_per_chunk':num_questions_per_chunk})
        question = result['question']
        answer = result['answer']
    else:
        continue
        
    df.loc[i, 'question'] = question
    df.loc[i, 'answer'] = answer
    if i==10:
        break


10it [00:50,  5.05s/it]


In [261]:
print(df.loc[3]['question'])

- 전기 자동차의 주요 장치 중 하나인 OBC는 무엇을 하는 장치인가요?
- 고전압 부품 및 구동용(고전압) 배터리를 분리하거나 손상시키면 어떤 위험성이 있을까요?


In [262]:
print(df.loc[3]['answer'])


- OBC는 구동용(고전압) 배터리를 충전하는 장치입니다.
- 고전압 부품 및 구동용(고전압) 배터리를 분리하거나 손상시키면 감전 등의 사고가 발생해 심각한 부상을 입을 수 있으며 차량의 성능 및 내구에 영향을 줄 수 있습니다. 
- 고전압 부품 및 구동용(고전압) 배터리의 점검 및 정비가 필요할 경우 당사 직영하이테크센터나 블루핸즈에서 점검을 받아야 합니다.


In [263]:
df[['h1', 'h2', 'h3', 'doc_contents', 'question', 'answer']].loc[:10].to_csv("~/test.csv",index=False, encoding='cp949')